In [5]:

import os as os
import pandas as pd

# what does this do?
# for the enrichment analysis,
# it is important to use an appropriate
# background list. Since chrY is removed
# from all datasets, create a corresponding
# list of "available" genes that can be used
# as custom background in GeneTrail

# the processed DESeq2 output files are already
# filtered in the right way, just merge two
# and dump the gene IDs
input_folder = '/TL/deep/fhgfs/projects/pebert/thesis/projects/statediff/deseq/bed_out'

input_diff = os.path.join(input_folder, 'deseq2_He_vs_Mo_diff_body.bed')
input_stable = os.path.join(input_folder, 'deseq2_He_vs_Mo_stable_body.bed')

deg = pd.read_csv(input_diff, sep='\t', header=0)

seg = pd.read_csv(input_stable, sep='\t', header=0)

genes = pd.concat([deg, seg], axis=0, ignore_index=False)

assert genes['name'].size == genes['name'].unique().size, 'Gene sets not disjoint'

genes['ens_id'] = genes['name'].str.extract('(?P<ENS>\w+)', expand=False)
genes['name'] = genes['ens_id']
genes['score'] = genes['end'] - genes['start']
genes['score'] = genes['score'].clip(lower=0, upper=1000, inplace=False)

genes.sort_values(by=['#chrom', 'start', 'end'], inplace=True)

ref_folder = '/TL/deep/fhgfs/projects/pebert/thesis/projects/statediff/references'
bglist = 'gencode_v21_pcg_bglist'

bed_out = os.path.join(ref_folder, bglist + '.bed')

genes.to_csv(bed_out, sep='\t', header=True, index=False,
             columns=['#chrom', 'start', 'end', 'name', 'score', 'strand', 'symbol'])

tsv_out = os.path.join(ref_folder, bglist + '.tsv')

genes.to_csv(tsv_out, sep='\t', header=False, index=False,
             columns=['name'])
